In [1]:
import requests
import os
import pandas as pd
import numpy

In [2]:
def get_api_id():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_ID":
                return listing[1]

def get_api_secret():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_KEY":
                return listing[1]


In [24]:
def get_EVAbyName(station_names):
    url = "https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/stop-places/by-name/"
    headers = {
                "DB-Client-ID": "c3ff7cbf26563615a6b0f0ef81fff3b6",
                "DB-Api-Key": "03408a082deb900e39719a0f7910f040",
                "accept": "application/vnd.de.db.ris+json"
            }
    eva_numbers = []

    for name in station_names:
        url = url + str(name)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            temp = []
            # get response
            temp.append(response.json())
            # now sort out that response is actually correct
            l = []
            for i in temp:
                for j in i['stopPlaces']:
                    l.append((j['evaNumber'], j['names']['DE']['nameLong']))
            # now remove all entries that are not correct:
            new_list=[li for li in l if li[1] == name]
            eva_numbers.append(new_list)
            
        else:
            print(f"Failed to fetch data for station {name}")
    
 
    # flatten
    return [x for xs in eva_numbers  for x in  xs]

In [28]:
eva_number = get_EVAbyName(["Frankfurt(Main)Hbf"])[0][0]
eva_number

'8000105'

In [38]:
def timetable(date, time, station_name, base_url="https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/"):
    eva_number = get_EVAbyName([station_name])[0][0]
    if(eva_number==[]):
        print(f"Failed to fetch station")
        return []
    headers = {
                "DB-Client-ID": "c3ff7cbf26563615a6b0f0ef81fff3b6",
                "DB-Api-Key": "03408a082deb900e39719a0f7910f040",
                "accept": "application/xml"
            }
    # time in HH
    # date in YYMMDD
    url = base_url + str(eva_number) + "/" + str(date) + "/" + str(time)
    #url='https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/8000105/240523/08'
    response = requests.get(url, headers=headers)
    print(response.text)
    if response.status_code == 200:
        timetable1=pd.read_xml(response.text, xpath=".//tl")
        timetable2=pd.read_xml(response.text, xpath=".//dp")
        timetable3=pd.read_xml(response.text, xpath=".//s")
        timetable = pd.concat([timetable1,timetable2,timetable3], axis=1)
    else:
        print(f"Failed to fetch timetable")
        timetable=[]
    return timetable

In [39]:
timetable("240523","09","Frankfurt(Main)Hbf")


Failed to fetch timetable


[]

In [90]:
type(get_EVAbyName("Frankfurt(Main)Hbf"))

list

In [4]:
api = ApiAuthentication(get_api_id().replace("\'", ""), get_api_secret().replace("\'", ""))
success: bool = api.test_credentials()

NameError: name 'ApiAuthentication' is not defined

In [ ]:
station_helper = StationHelper()
found_stations = station_helper.find_stations_by_lat_long(47.996713, 7.842174, 10)
found_stations_by_name = station_helper.find_stations_by_name("Freiburg")

In [ ]:
timetable_helper = TimetableHelper(YOUR_STATION_OBJECT, YOUR_API_AUTHENTICATION_OBJECT)
trains_in_this_hour = timetable_helper.get_timetable()
trains_at_given_hour = timetable_helper.get_timetable(12)

In [ ]:
trains_with_changes = timetable_helper.get_timetable_changes(trains)